# Measure contributions of parts of OCR pipeline

## Possible OCR pipelines (actions are green)

![ocr_flow](assets/ocr_flow.drawio.jpg)

#### The are 4 groups of actions in the full pipeline (green boxes). The purpose of this notebook is to test if all of these actions really helps with the OCR results, and if they do, by how much.

- Which of the 4 image processing pipelines improve OCR performance?
- Two OCR engines: `Tesseract` & `EasyOCR`. `Tesseract` is the current leader in open source OCR engines, does adding `EasyOCR` improve the results?
- The `combine text` function is only needed if we stick with the ensemble approach. I.e. only if we use more than one image processing pipeline or more than one OCR engine.
- The `clean text` function corrects misspellings and common OCR errors with punctuation, spacing, etc. We want to measure its efficacy.

## Comparison strategy

#### We're doing ablations on the OCR pipeline.
- How well do `Tesseract` and `EasyOCR` perform on their own without image pre-processing. I'll also try the engine directly grafted to the `clean text` function.
- How well do each of the image pre-processing steps help the OCR process? and which ones work well with which OCR engine. I'm going to try various permutations of these.
- Can I whittle this down to one or zero image pre-processing pipelines and one OCR engine? If so, then this would allow me to drop the `combine text` step.
- How much does the `clean text` step help?
- Note that EasyOCR uses a fair bit of GPU resources and if we can remove it it will speed up the OCR pipeline significantly. I.e. it's difficult to parallelize.

#### Scoring
- I'll use an expert derived gold standard to compare against the ablation sequences.
- I am using Levenshtein distance as the scores. Levenshtein distance counts character mismatches between sequences in a best case pairwise alignment.
- I will normalize the Levenshtein distance it by dividing by the length of the gold standard sequence.

## Setup

In [11]:
import json
from collections import defaultdict
from itertools import groupby
from pathlib import Path
from types import SimpleNamespace

from digi_leap.pylib import consts
from digi_leap.pylib.ocr import ocr_compare as comp

## Constants

In [2]:
ARGS = SimpleNamespace(
    database=Path(consts.DATA_DIR / "sernec" / "sernec.sqlite"),
    gold_set="test_gold_set",
    score_set="notebook_scores",
    notes="",
    csv_path=consts.DATA_DIR / "sernec" / "gold_std_2022-06-28_sample.csv",
)

## Get a gold standard

In [3]:
# Save a new gold standard to a database

# comp.insert_gold_std(ARGS.csv_path, ARGS.database, ARGS.gold_set)

In [4]:
# Read a gold standard for the database

GOLD_STD = comp.select_gold_std(ARGS.database, ARGS.gold_set)
GOLD_DICT = {g["gold_id"]: g for g in GOLD_STD}

## Get OCR scores

In [5]:
scorer = comp.Scorer(ARGS)

In [6]:
# Calculate new scores

# SCORES = scorer.calculate(GOLD_STD)
# scorer.insert_scores(SCORES)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:28<00:00, 22.11s/it]


In [7]:
SCORES = scorer.select_scores()

## Peek at the scores

In [8]:
def peek_scores(scores):
    grouped_scores = groupby(scores, key=lambda s: s["label_id"])

    for labels_id, scores in grouped_scores:
        print("=" * 80)
        print(labels_id)
        scores = sorted(scores, key=lambda s: (s["score"], len(s["actions"])))
        for score in scores:
            print(f"{score['score']:4d}  {score['actions']}")


# peek_scores(SCORES)

In [12]:
def normalized_scores(scores, gold_std):
    tally = defaultdict(float)

    for score in scores:
        gold = gold_std[score["gold_id"]]
        denominator = float(len(gold["gold_text"]))
        tally[score["actions"]] += score["score"] / denominator

    tally = [(v, len(json.loads(k)), k) for k, v in tally.items()]
    tally = sorted(tally)

    norms = [f"{s:10.4f}  {a}" for s, _, a in tally]
    return norms


norms = normalized_scores(SCORES, GOLD_DICT)

for norm in norms:
    print(norm)

    0.1734  [["", "tesseract"], ["deskew", "tesseract"], ["binarize", "tesseract"], ["denoise", "tesseract"], ["post_process"]]
    0.1741  [["deskew", "easyocr"], ["deskew", "tesseract"], ["binarize", "easyocr"], ["binarize", "tesseract"], ["post_process"]]
    0.1752  [["", "easyocr"], ["", "tesseract"], ["deskew", "easyocr"], ["deskew", "tesseract"], ["post_process"]]
    0.1760  [["", "tesseract"], ["binarize", "tesseract"], ["denoise", "tesseract"], ["post_process"]]
    0.1864  [["deskew", "tesseract"], ["binarize", "tesseract"], ["denoise", "tesseract"], ["post_process"]]
    0.2096  [["", "tesseract"], ["deskew", "tesseract"], ["binarize", "tesseract"], ["post_process"]]
    0.2098  [["binarize", "easyocr"], ["binarize", "tesseract"], ["denoise", "easyocr"], ["denoise", "tesseract"], ["post_process"]]
    0.2131  [["deskew", "easyocr"], ["deskew", "tesseract"], ["post_process"]]
    0.2159  [["", "tesseract"], ["deskew", "tesseract"], ["denoise", "tesseract"], ["post_process"]]

In [10]:
len(norms)

90